# Aula 4 - Exercícios Extra - Pandas, Fontes de dados e Agregação
### Consulta a internet (Google, Stackoverflow e etc) liberada!!

In [1]:
# Importe as bibliotecas pandas, numpy, glob e sqlalchemy
import pandas as pd
import numpy as np


### 4.9 Construa uma função para concatenar (verticalmente) o conteúdo de vários arquivos CSV do dataset de morbidade. Esses arquivos contêm dados com colunas compatíveis.
#### Extraia a data do cabeçalho do arquivo. Divida a coluna 'Município' em duas outras contendo o código ibge de 6 caracteres e o nome do município.



In [2]:
# Dica 1: Construa uma lista de arquivos presentes num diretório usando a função glob da biblioteca glob
# Dica 2: Num loop, leia os arquivo duas vezes. Na primeira vez, pra carregar somente a parte desejada do cabeçalho,
#         onde se encontra o mês/ano do respectivo arquivo, utilize o parâmetro nrows. Preste atenção ao separador 
#         dos arquivos CSV, é um ponto e vírgula.
# Dica 3: O método split pode dividir uma string na posição de um caracter especificado.
# Dica 4: Se quiser converter o nome do mês para o seu número, você precisa ajustar a configuração de locale
#         do python para 'pt_BR'. Use o código a seguir para fazer esta operação.
#            import locale
#            locale.setlocale(locale.LC_ALL, 'pt_BR')
# Dica 5: O código strptime(nome_mes,'%b').tm_mon converte nome do mês no seu respectivo número de 1 a 12
# Dica 6: Para criar uma data a partir dos seus atributos use um código semelhante ao exemplo a seguir
#            import datetime
#            datetime.date(year=2020, month=1, day=1)
# Dica 7: Para construir uma nova coluna aplicando uma regra para cada célula use a função apply e construa uma
#         função lambda.
# Dica 8: O método join de uma string consegue unir, com um caracter separador, o conteúdo de uma lista de 
#         elementos numa string. Exemplo: ' '.join(lista_exemplo), produz uma string concatenando com um espaçao
#         os elementos da lista_exemplo. Pode ser útil para isolar o nome do município.
# Dica 9: Adicione todos os dataframes construídos dentro do loop a uma lista e depois use o método concat para 
#         concatenar os dataframes verticalmente. Use o parâmetro ignore_index=True para ignorar o índice dos 
#         dataframes.

def concat_morbidade(files_path):
    """Retorne um dataframe com a concatenação vertical dos dados dos arquivos do diretorio dir_path
    """
    import glob
    from time import strptime
    import datetime
    import locale
    locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')
    
    partes = []
    for file in glob.glob(files_path):
        periodo = pd.read_csv(file, encoding='latin1', skiprows=2, nrows=1)
        mes_ano = periodo.columns.values[0].split(':')[1]
        mes, ano = mes_ano.split('/')
        frame = pd.read_csv(file, encoding='latin1', skiprows=3, sep=';', skipfooter=7, engine='python')
        frame['mes'] = mes
        frame['ano'] = ano
        frame['mes_ano'] = datetime.date(year=int(ano), month=strptime(mes,'%b').tm_mon, day=1)
        frame['cod_ibge6'] = frame['Município'].apply(lambda x: x.split(' ')[0]) 
        frame['Município'] = frame['Município'].apply(lambda x: ' '.join(x.split(' ')[1:])) 
        partes.append(frame)
        
    # Concatena tudo num único DataFrame    
    return pd.concat(partes, ignore_index=True)


In [3]:
# Solução 2, usando pandas.DataFrame.append
def concat_morbidade(files_path):
    """Retorne um dataframe com a concatenação vertical dos dados dos arquivos do diretorio dir_path
    """
    import glob
    from time import strptime
    import datetime
    import locale
    locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')
    
    ret = pd.DataFrame()
    for file in glob.glob(files_path):
        periodo = pd.read_csv(file, encoding='latin1', skiprows=2, nrows=1)
        mes_ano = periodo.columns.values[0].split(':')[1]
        mes, ano = mes_ano.split('/')
        frame = pd.read_csv(file, encoding='latin1', skiprows=3, sep=';', skipfooter=7, engine='python')
        frame['mes'] = mes
        frame['ano'] = ano
        frame['mes_ano'] = datetime.date(year=int(ano), month=strptime(mes,'%b').tm_mon, day=1)
        frame['cod_ibge6'] = frame['Município'].apply(lambda x: x.split(' ')[0]) 
        frame['Município'] = frame['Município'].apply(lambda x: ' '.join(x.split(' ')[1:])) 
        ret = ret.append(frame, ignore_index=True)
    
    return ret


In [4]:
## Faça seus testes aqui
files_path = '../../curso_ciencia_dados2020/data/originais/morbidade/2020/*.csv'



In [5]:
# Validação para nota
files_path = '../../curso_ciencia_dados2020/data/originais/morbidade/2020/*.csv'
test_data = set(['10899,59', '16793,57', '6585,96', '7446,97', '19054,46', '11462,34'])
morbidade = concat_morbidade(files_path)
slice_dict = set(morbidade.loc[morbidade['cod_ibge6']=='522230', 'Valor_total'].to_dict().values())

assert slice_dict == test_data